# YOLOv8 Object Tracking Training

This notebook demonstrates tracking with YOLOv8 + StrongSORT and fine-tuning tracking models.

## Requirements
- ultralytics
- boxmot (for StrongSORT)
- PyTorch with CUDA support

In [ ]:
# Import required libraries
from ultralytics import YOLO
import torch
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Load Detection Model for Tracking

Tracking uses a detection model + tracking algorithm.

In [ ]:
# Load YOLOv8 detection model for tracking
model_path = '../models/tracking/yolo/yolov8s.pt'
model = YOLO(model_path)

print(f"Model loaded: {model_path}")
print(f"Model task: {model.task}")

## 2. Basic Tracking with YOLOv8

YOLOv8 has built-in tracking using BoT-SORT and ByteTrack.

In [ ]:
# Track objects in a video
# results = model.track(
#     source='path/to/video.mp4',
#     show=True,
#     save=True,
#     tracker='botsort.yaml',  # or 'bytetrack.yaml'
#     conf=0.3,
#     iou=0.5,
#     device=0,
#     project='runs/track',
#     name='yolov8_tracking',
# )

# Available trackers:
trackers_info = """
Built-in YOLOv8 Trackers:
1. BoT-SORT (botsort.yaml) - Better for crowded scenes
2. ByteTrack (bytetrack.yaml) - Faster, good for real-time

External Trackers (with boxmot):
3. StrongSORT - Most accurate, uses Re-ID
4. DeepOCSORT - Balance of speed and accuracy
5. OCSORT - Fast, simple
"""

print(trackers_info)

## 3. StrongSORT Tracking

StrongSORT provides better tracking with Re-ID features.

In [ ]:
# Install boxmot for StrongSORT
# !pip install boxmot

try:
    from boxmot import StrongSORT
    print("✓ BoxMOT (StrongSORT) is available")
except ImportError:
    print("⚠️ BoxMOT not installed. Run: pip install boxmot")

In [ ]:
# StrongSORT tracking configuration
strongsort_config = {
    'reid_model_path': '../models/tracking/strongsort/osnet_x0_25_msmt17.pt',
    'device': 'cuda:0',
    'fp16': True,  # Use half precision
    'max_dist': 0.2,  # Maximum cosine distance
    'max_iou_distance': 0.7,  # Maximum IOU distance
    'max_age': 30,  # Maximum frames to keep alive a track
    'n_init': 3,  # Number of frames to confirm a track
    'nn_budget': 100,  # Maximum size of feature bank
}

print("StrongSORT Configuration:")
for key, value in strongsort_config.items():
    print(f"  {key}: {value}")

In [ ]:
# Use StrongSORT with YOLOv8
# def track_with_strongsort(video_path, model):
#     from boxmot import StrongSORT
#     
#     # Initialize StrongSORT
#     tracker = StrongSORT(
#         model_weights=Path('../models/tracking/strongsort/osnet_x0_25_msmt17.pt'),
#         device='cuda:0',
#         fp16=True,
#     )
#     
#     # Open video
#     cap = cv2.VideoCapture(video_path)
#     
#     # Get video properties
#     fps = int(cap.get(cv2.CAP_PROP_FPS))
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     
#     # Output video writer
#     out = cv2.VideoWriter(
#         'output_strongsort.mp4',
#         cv2.VideoWriter_fourcc(*'mp4v'),
#         fps, (width, height)
#     )
#     
#     frame_idx = 0
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         
#         # Run detection
#         results = model(frame, verbose=False)
#         
#         # Get detections
#         if len(results[0].boxes) > 0:
#             dets = results[0].boxes.xyxy.cpu().numpy()
#             confs = results[0].boxes.conf.cpu().numpy()
#             class_ids = results[0].boxes.cls.cpu().numpy()
#             
#             # Update tracker
#             tracks = tracker.update(dets, confs, class_ids, frame)
#             
#             # Draw tracks
#             for track in tracks:
#                 x1, y1, x2, y2, track_id, conf, cls = track
#                 cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
#                 cv2.putText(frame, f'ID: {int(track_id)}', (int(x1), int(y1) - 10),
#                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
#         
#         out.write(frame)
#         frame_idx += 1
#         
#         if frame_idx % 100 == 0:
#             print(f"Processed {frame_idx} frames")
#     
#     cap.release()
#     out.release()
#     print(f"\n✓ Tracking complete! Output saved to output_strongsort.mp4")

# Example usage:
# track_with_strongsort('path/to/video.mp4', model)

print("StrongSORT tracking function ready.")

## 4. Train Custom Tracking Dataset

For tracking, you typically train a better detection model.

In [ ]:
# Training parameters for tracking-optimized detection
training_params = {
    'data': 'path/to/data.yaml',
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,
    'device': 0,
    'project': 'runs/detect',
    'name': 'yolov8_tracking_optimized',
    'patience': 50,
    # Tracking-specific optimizations
    'conf': 0.25,  # Lower confidence for better recall
    'iou': 0.45,
    'augment': True,  # Important for tracking robustness
    'mosaic': 1.0,  # Use mosaic augmentation
    'mixup': 0.1,  # Use mixup augmentation
}

print("Tracking-optimized training parameters:")
for key, value in training_params.items():
    print(f"  {key}: {value}")

In [ ]:
# Train model for tracking
# results = model.train(**training_params)

print("Training ready. Uncomment to start.")

## 5. Evaluate Tracking Performance

Tracking metrics: MOTA, MOTP, IDF1, etc.

In [ ]:
# Tracking evaluation metrics
tracking_metrics = """
Key Tracking Metrics:

1. MOTA (Multiple Object Tracking Accuracy)
   - Overall tracking accuracy considering FP, FN, ID switches
   - Range: -∞ to 1 (higher is better)

2. MOTP (Multiple Object Tracking Precision)
   - Average distance between predictions and ground truth
   - Lower is better

3. IDF1 (ID F1 Score)
   - Ratio of correctly identified detections
   - Range: 0 to 1 (higher is better)

4. ID Switches
   - Number of times track IDs change
   - Lower is better

5. Fragmentation
   - Number of times a track is interrupted
   - Lower is better
"""

print(tracking_metrics)

## 6. Advanced Tracking Techniques

Tips for better tracking performance.

In [ ]:
# Best practices for tracking
best_practices = """
Tracking Optimization Tips:

1. Detection Quality:
   - Use higher resolution (640-1280)
   - Lower confidence threshold (0.2-0.3) for better recall
   - Train on domain-specific data

2. Tracker Selection:
   - ByteTrack: Fast, real-time applications
   - BoT-SORT: Crowded scenes, occlusions
   - StrongSORT: Best accuracy, Re-ID features

3. Re-ID Model:
   - OSNet: General purpose
   - ResNet: More accurate but slower
   - MobileNet: Faster but less accurate

4. Hyperparameters:
   - max_age: Increase for slow-moving objects
   - n_init: Decrease for fast initialization
   - iou_threshold: Adjust based on object overlap

5. Post-processing:
   - Track smoothing (Kalman filter)
   - Interpolation for missing frames
   - Track merging for split tracks
"""

print(best_practices)

## 7. Export Tracking Model

Export optimized model for deployment.

In [ ]:
# Export to TensorRT for faster tracking
# tensorrt_model = model.export(
#     format='engine',
#     device=0,
#     half=True,
#     workspace=4,
# )

# Save to models directory
# import shutil
# tensorrt_dest = Path('../models/tensorrt/tracking/')
# tensorrt_dest.mkdir(parents=True, exist_ok=True)
# shutil.copy(tensorrt_model, tensorrt_dest / 'yolov8s_tracking.engine')

print("Export ready. Uncomment after training.")

## 8. Real-time Tracking Demo

Process webcam or RTSP stream.

In [ ]:
# Real-time tracking from webcam
# def realtime_tracking():
#     cap = cv2.VideoCapture(0)  # Webcam
#     
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
#         
#         # Track objects
#         results = model.track(frame, persist=True, verbose=False)
#         
#         # Visualize
#         annotated = results[0].plot()
#         
#         cv2.imshow('YOLOv8 Tracking', annotated)
#         
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     
#     cap.release()
#     cv2.destroyAllWindows()

# Example usage:
# realtime_tracking()

print("Real-time tracking function ready.")

## Summary

This notebook covers:
1. ✅ Loading YOLOv8 models for tracking
2. ✅ Built-in tracking (BoT-SORT, ByteTrack)
3. ✅ StrongSORT integration with Re-ID
4. ✅ Training tracking-optimized detectors
5. ✅ Tracking evaluation metrics
6. ✅ Best practices and optimization tips
7. ✅ Model export for deployment
8. ✅ Real-time tracking implementation

### Next Steps
- Download StrongSORT Re-ID model
- Test different trackers on your videos
- Fine-tune detection model for your domain
- Optimize hyperparameters for your use case
- Integrate with AppStore tracking app